Define and train the network based on the dataset

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)

dataset_directory = 'dataset_final'
image_size = (120, 120) # (height, width)
batch_size = 64
classes = sorted(os.listdir(dataset_directory))

print(classes)

training_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    seed=1234,
    validation_split=0.2,
    subset='training')

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_directory,
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=image_size,
    seed=1234,
    validation_split=0.2,
    subset='validation')

training_dataset = training_dataset.cache().prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(tf.data.AUTOTUNE)


model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255., input_shape=(image_size[0], image_size[1], 1)),

    tf.keras.layers.Conv2D(16, 3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(32, 3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(64, 3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(128, 3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(256, 3, padding='same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(len(classes))
])

model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(training_dataset, validation_data=validation_dataset, epochs=100, callbacks=[callback])

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

model.save('classification_model_final_d.hdf5')

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

dataset_directory = 'dataset_final'
image_filename = 'image.png'

img = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (120, 120))
#img = cv2.bitwise_not(img)

plt.figure()
plt.imshow(img, cmap='gray')

probability_model = tf.keras.models.load_model('classification_model_final.hdf5')
#probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

probability_model.summary()

img_batch = np.array([img])

probabilities = probability_model.predict(img_batch)

classes = sorted(os.listdir(dataset_directory))
prob_dict = {}
for i in range(len(classes)):
    prob_dict[classes[i]] = probabilities[0][i]

class_index = np.argmax(probabilities)
print(classes[class_index], 'probability:', probabilities[0][class_index])
import pprint
pprint.pprint(prob_dict)

Add the `Softmax` layer to the model

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('classification_model_final_c.hdf5')
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

probability_model.save('classification_model_final_c_prob.hdf5')

Convert the model to TFLite

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('classification_model_final_c_prob.hdf5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('classification_model_final_c_prob.tflite', 'wb') as f:
    f.write(tflite_model)